In [1]:
print('\nInitializing 2ES Target Selection Pipeline...')

from config import *
from core.gaia_queries import *
from core.data_processing import *
from analysis.plotting import *
from calculations.stellar_calculations import *
from core.utils import *
from analysis.catalog_integration import CatalogProcessor, add_rhk_to_dataframe
from analysis.filtering import filter_stellar_data
from analysis.gaia_tess_overlap import run_tess_overlap_batch
from analysis.HWO_overlap import HWO_match
from analysis.plato_lops2 import plato_lops2_match
import matplotlib.pyplot as plt


# -----------------------------------------------------------
# Gaia queries and merging
# -----------------------------------------------------------
df_dr2 = execute_gaia_query(
    get_dr2_query(),
    str_columns=['source_id'],
    output_file=f"{RESULTS_DIRECTORY}dr2_results.xlsx"
)

dr2_source_ids = tuple(df_dr2['source_id'])
df_crossmatch = execute_gaia_query(
    get_crossmatch_query(dr2_source_ids),
    str_columns=['dr2_source_id', 'dr3_source_id'],
    output_file=f"{RESULTS_DIRECTORY}dr2_dr3_crossmatch.xlsx"
)

df_dr3 = execute_gaia_query(
    get_dr3_query(),
    str_columns=['source_id'],
    output_file=f"{RESULTS_DIRECTORY}dr3_results.xlsx"
)

merged_results = process_gaia_data(df_dr2, df_dr3, df_crossmatch)
clean_results = clean_merged_results(merged_results)
consolidated_results = consolidate_data(clean_results)

# -----------------------------------------------------------
# External catalog enrichment
# -----------------------------------------------------------
processor = CatalogProcessor(
    celesta_path='../data/Catalogue_CELESTA.txt',
    stellar_catalog_path='../data/Catalogue_V_117A_table1.txt'
)
df_consolidated = processor.process_catalogs(consolidated_results)

plot_density_vs_logg(
    df_consolidated,
    output_path=f'{FIGURES_DIRECTORY}density_vs_logg.png',
    show_plot=False
)

df_consolidated = add_rhk_to_dataframe(df_consolidated)
save_and_adjust_column_widths(df_consolidated, '../results/merged_df_with_rhk.xlsx')

# -----------------------------------------------------------
# Filtering + color-magnitude diagram
# -----------------------------------------------------------
df_filtered = filter_stellar_data(df_consolidated, STELLAR_FILTERS)

plot_color_histogram(
    df_filtered,
    output_path=f'{FIGURES_DIRECTORY}color_histogram.png',
    show_plot=False
)

combined_df = plot_color_magnitude_diagram(
    df_filtered,
    output_path=f'{FIGURES_DIRECTORY}color_magnitude_diagram.png',
    show_plot=False
)

df_with_bright_neighbors, df_without_bright_neighbors = analyze_bright_neighbors(
    merged_df=combined_df,
    search_radius=SEARCH_RADIUS,
    execute_gaia_query_func=execute_gaia_query
)

df = df_without_bright_neighbors.copy()

# -----------------------------------------------------------
# Habitable zone & noise modeling
# -----------------------------------------------------------
df = calculate_and_insert_habitable_zone(df)
df = calculate_and_insert_photon_noise(df)
df = calculate_hz_orbital_period(df)
df = add_granulation_to_dataframe(df)
df = add_pmode_rms_to_dataframe(df)
df = calculate_and_insert_RV_noise(df)
df = calculate_K(df)
df = calculate_K(df, sigma_rv_col='σ_RV,total [m/s]')
df = calculate_and_insert_hz_detection_limit(
    df,
    semi_amplitude_col='K_σ_photon [m/s]'
)
df = calculate_and_insert_hz_detection_limit(
    df,
    semi_amplitude_col='K_σ_RV_total [m/s]'
)

# -----------------------------------------------------------
# Plots
# -----------------------------------------------------------
plot_scatter(
    x='RA',
    y='DEC',
    data=df,
    xlabel='Right Ascension (RA)',
    ylabel='Declination (DEC)',
    xlim=(0, 360),
    filename=f'{FIGURES_DIRECTORY}ra_dec.png',
    alpha=0.6,
    invert_xaxis=True,
    show_plot=False
)

plt.figure(figsize=(10, 8), dpi=150)
plt.scatter(
    df['T_eff [K]'],
    df['Luminosity [L_Sun]'],
    c=df['T_eff [K]'],
    cmap='autumn',
    alpha=0.99,
    edgecolors='w',
    linewidths=0.05,
    s=df['Radius [R_Sun]'] * 20
)
plt.colorbar(label='Effective Temperature (K)')
plt.xscale('log')
plt.yscale('log')
plt.xlim(min(df['T_eff [K]']) - 50, max(df['T_eff [K]']) + 50)
plt.ylim(min(df['Luminosity [L_Sun]']), max(df['Luminosity [L_Sun]']) + 0.5)
plt.gca().invert_xaxis()
plt.xlabel('Effective Temperature [K]')
plt.ylabel('Luminosity [L/L_sun]')
plt.title('Hertzsprung-Russell Diagram')
plt.grid(True, which="both", ls="--", linewidth=0.5)
plt.savefig(f'{FIGURES_DIRECTORY}HR_diagram.png')
plt.close()

for detection_limit in DETECTION_LIMITS:
    plot_hr_diagram_with_detection_limit(
        df,
        use_filtered_data=detection_limit is not None,
        detection_limit=detection_limit
    )


Initializing 2ES Target Selection Pipeline...

Querying stars from Gaia DR2
2161 stars retrieved
Query results saved to ../results/dr2_results.xlsx

Crossmatching stellar IDs between Gaia DR2 and DR3
2268 stars retrieved
Query results saved to ../results/dr2_dr3_crossmatch.xlsx

Querying stars from Gaia DR3
2093 stars retrieved
Query results saved to ../results/dr3_results.xlsx

Merging Gaia DR2 and DR3 data with crossmatch information
Results saved to ../results/merged_results.xlsx

Cleaning merged results by removing duplicate dr2_source_id entries
Results saved to ../results/repeated_entries.xlsx

Processing repeated entries with the same dr2_source_id
Results saved to ../results/clean_merged_results.xlsx
Results saved to ../results/removed_rows.xlsx
Original shape of merged_results: (2330, 29)
Shape after removing duplicates: (2222, 29)
Number of rows removed: 108

Merging and consolidating Gaia DR2 and DR3 data columns...
Merging the following columns between DR2 and DR3: ra, dec

Retrieving HD, GJ and HIP numbers and object type from Simbad based on Gaia identifiers:  12%|█▏        | 261/2222 [06:14<26:37,  1.23it/s]  

Attempt 1 failed for Gaia DR3 2648705785983069056: ProxyError: HTTPSConnectionPool(host='simbad.cds.unistra.fr', port=443): Max retries exceeded with url: /simbad/sim-tap/sync (Caused by ProxyError('Unable to connect to proxy', RemoteDisconnected('Remote end closed connection without response')))
Attempt 2 failed for Gaia DR3 2648705785983069056: ConnectionError: HTTPSConnectionPool(host='simbad.cds.unistra.fr', port=443): Max retries exceeded with url: /simbad/sim-tap/sync (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x12b038410>: Failed to resolve 'simbad.cds.unistra.fr' ([Errno 8] nodename nor servname provided, or not known)"))


Retrieving HD, GJ and HIP numbers and object type from Simbad based on Gaia identifiers:  12%|█▏        | 262/2222 [22:01<154:50:26, 284.40s/it]

Attempt 3 failed for Gaia DR3 2648705785983069056: ConnectionError: HTTPSConnectionPool(host='simbad.cds.unistra.fr', port=443): Max retries exceeded with url: /simbad/sim-tap/sync (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x12b038690>: Failed to resolve 'simbad.cds.unistra.fr' ([Errno 8] nodename nor servname provided, or not known)"))
All 3 attempts failed for Gaia DR3 2648705785983069056
Attempt 1 failed for Gaia DR3 2648914040357320576: ConnectionError: HTTPSConnectionPool(host='simbad.cds.unistra.fr', port=443): Max retries exceeded with url: /simbad/sim-tap/sync (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x12b038cd0>: Failed to resolve 'simbad.cds.unistra.fr' ([Errno 8] nodename nor servname provided, or not known)"))
Attempt 2 failed for Gaia DR3 2648914040357320576: ConnectionError: HTTPSConnectionPool(host='simbad.cds.unistra.fr', port=443): Max retries exceeded with url: /simbad/sim-tap/sync (Caused by Nam

...

Parallel processing for detecting bright neighbors:  17%|█     | 216/1269 [14:31<1:44:57,  5.98s/it]

An error occurred: [Errno 54] Connection reset by peer
Retrying in 5 seconds... (Attempt 1/3)
An error occurred: [Errno 54] Connection reset by peer
Retrying in 5 seconds... (Attempt 1/3)
An error occurred: [Errno 54] Connection reset by peer
Retrying in 5 seconds... (Attempt 1/3)
An HTTP error occurred: 500
Retrying in 5 seconds... (Attempt 1/3)
An error occurred: [Errno 54] Connection reset by peer
Retrying in 5 seconds... (Attempt 1/3)
An HTTP error occurred: 500
Retrying in 5 seconds... (Attempt 1/3)
An HTTP error occurred: 500
Retrying in 5 seconds... (Attempt 1/3)
An error occurred: [Errno 54] Connection reset by peer
Retrying in 5 seconds... (Attempt 1/3)
An error occurred: [Errno 54] Connection reset by peer
Retrying in 10 seconds... (Attempt 2/3)
An error occurred: [Errno 60] Operation timed out
Retrying in 20 seconds... (Attempt 3/3)


Parallel processing for detecting bright neighbors: 100%|███████| 1269/1269 [58:37<00:00,  2.77s/it]

Failed to execute query after several attempts.
Stars with bright neighbors: 50
Stars without bright neighbors: 1219
Results saved to ../results/stars_with_bright_neighbors.xlsx


Results saved to ../results/stars_without_bright_neighbors.xlsx

Calculating and inserting habitable zone limits
Results saved to ../results/consolidated_results.xlsx

Calculating and inserting RV precision for each star
Results saved to ../results/combined_query_with_RV_precision.xlsx

Calculating minimum detectable RV semi-amplitude K

Calculating minimum detectable RV semi-amplitude K

Calculating and inserting habitable zone detection limits using 'K_σ_photon [m/s]' -> 'HZ Detection Limit (σ_photon) [M_Earth]'
Number of NaN values in HZ Detection Limit (σ_photon) [M_Earth]: 0
Results saved to ../results/combined_query_with_mass_detection_limit.xlsx

Calculating and inserting habitable zone detection limits using 'K_σ_RV_total [m/s]' -> 'HZ Detection Limit [M_Earth]'
Number of NaN values in HZ Detection Limit [M_Earth]: 0
Results saved to ../results/combined_query_with_mass_detection_limit.xlsx
Saved ../figures/ra_dec.png

Plotting HR diagram with detection limit
Saved ../figures/HR

In [13]:
list(df.columns)

['source_id',
 'source_id_dr2',
 'source_id_dr3',
 'RA',
 'DEC',
 'V_mag',
 'Phot G Mean Mag',
 'Phot BP Mean Mag',
 'Phot RP Mean Mag',
 'BP-RP',
 'Parallax',
 'T_eff [K]',
 'Mass [M_Sun]',
 'Luminosity [L_Sun]',
 'Radius [R_Sun]',
 'HZ_limit [AU]',
 'Orbital Period [days]',
 'Yearly_Observable_nights',
 'K_σ_RV_total [m/s]',
 'HZ Detection Limit [M_Earth]',
 'K_σ_photon [m/s]',
 'HZ Detection Limit (σ_photon) [M_Earth]',
 'σ_photon [m/s]',
 'σ_granulation [m/s]',
 'σ_p-mode [m/s]',
 'σ_RV,total [m/s]',
 'Density [Solar unit]',
 'logg_gaia',
 'log_rhk',
 'activity_noise_floor [m/s]',
 'rhk_source',
 'Gaia Spectral type',
 'HD Number',
 'GJ Number',
 'HIP Number',
 'Object Type',
 'Stellar Parameter Source',
 'SIMBAD Spectral Type',
 'Readable Spectral Type (experimental)']

In [227]:
def plot_stellar_properties_vs_temperature(df, detection_limit, show_plot=True, n_vmag_groups=None):
    """
    Plots various stellar properties vs. effective temperature as scatter plots with histograms,
    but histograms are for the whole data set (not split by V magnitude groups).
    Now includes distance (calculated from parallax).

    Parameters:
    df (DataFrame): DataFrame containing stellar data with columns for effective temperature, V_mag and other properties.
    detection_limit (float): The current detection limit used for filtering. (for filename)
    show_plot (bool): Whether to show the plot interactively.
    n_vmag_groups (int or None): Unused (API compatibility).
    """
    import matplotlib.gridspec as gridspec
    import numpy as np
    from matplotlib.lines import Line2D

    print('\nPlotting stellar properties vs temperature (Vmag color groups, histograms not group-split)')

    # --- GAIA COLOUR PALETTE from context
    COLOR_PALETTE = [
        "#e41a1c",
        "#ff7f00",
        "#4daf4a",
        "#377eb8",
        "#984ea3"
    ]
    N_COLOR_GROUPS = 5

    # Calculate distance from parallax if available, add as a column for plotting
    # Distance [pc] = 1000 / parallax(mas)
    if "Parallax" in df.columns and "Distance [pc]" not in df.columns:
        with np.errstate(divide='ignore', invalid='ignore'):
            distance_pc = 1000. / df["Parallax"].values
            # Set negative and zero parallaxes as NaN for plotting
            distance_pc[(df["Parallax"].values <= 0) | ~np.isfinite(df["Parallax"].values)] = np.nan
        df = df.copy()
        df["Distance [pc]"] = distance_pc

    columns = [
        'T_eff [K]',
        'V_mag',
        'Distance [pc]',        
        'log_rhk',        
        'Luminosity [L_Sun]',
        'Mass [M_Sun]',
        'Radius [R_Sun]',
        'Density [Solar unit]',
        'σ_photon [m/s]',
        'σ_p-mode [m/s]',
        'σ_granulation [m/s]',
        'σ_RV,total [m/s]',
        'K_σ_RV_total [m/s]',        
        'HZ_limit [AU]',
        'Orbital Period [days]',
        'HZ Detection Limit [M_Earth]',
    ]

    n_cols = 4
    n_rows = int(np.ceil(len(columns) / n_cols))

    fig = plt.figure(figsize=(3.6 * n_cols, 3 * n_rows), dpi=300)
    # Tighter subplots: decrease left/right/top/bottom and space between
    gs = gridspec.GridSpec(
        n_rows,
        n_cols * 2,
        width_ratios=[1, 3] * n_cols,
        left=0.01,
        right=0.99,
        top=0.93,
        bottom=0.04,
        wspace=0,
        hspace=0.19   # decreased from 0.34 to tighten vertical spacing
    )

    ylabels_dict = {
        'T_eff [K]': r'$T_\mathrm{eff}$ [K]',
        'V_mag': 'V Magnitude',
        'Distance [pc]': r'Distance [pc]',        
        'Mass [M_Sun]': r'Mass [$M_\odot$]',
        'Luminosity [L_Sun]': r'Luminosity [$L_\odot$]',
        'Radius [R_Sun]': r'Radius [$R_\odot$]',
        'Density [Solar unit]': r'Density [$\rho_\odot$]',
        'σ_photon [m/s]': r'$\sigma_\mathrm{photon}$ [m/s]',
        'σ_granulation [m/s]': r'$\sigma_\mathrm{gran}$ [m/s]',
        'σ_p-mode [m/s]': r'$\sigma_\mathrm{osc}$ [m/s]',
        'σ_RV,total [m/s]': r'$\sigma_\mathrm{RV}$ [m/s]',
        'K_σ_RV_total [m/s]': 'K [m/s]',        
        'log_rhk': r"$\log R'_\mathrm{HK}$",
        'HZ_limit [AU]': 'HZ [AU]',
        'Orbital Period [days]': 'P [days]',
        'HZ Detection Limit [M_Earth]': r'Detection Limit [$M_\oplus$]',
    }

    plot_axes = []
    hist_axes = []

    # Prepare V_mag groups and bin edges for their use in scatter colours
    if 'V_mag' in df.columns:
        finite_vmag = df['V_mag'].dropna().values
        Vmin, Vmax = np.min(finite_vmag), np.max(finite_vmag)
        # N_COLOR_GROUPS as in context
        vmag_edges = np.linspace(Vmin, Vmax, N_COLOR_GROUPS+1)
        df_color_group = np.digitize(df['V_mag'], bins=vmag_edges, right=False) - 1  # 0-based
        # Fix for possible rightmost group (which may produce index == N_COLOR_GROUPS)
        df_color_group[df_color_group == N_COLOR_GROUPS] = N_COLOR_GROUPS - 1
    else:
        df_color_group = np.zeros(len(df), dtype=int)
        vmag_edges = np.zeros(N_COLOR_GROUPS+1)

    for i, col in enumerate(columns):
        row = i // n_cols
        col_pos = i % n_cols
        hist_ax = fig.add_subplot(gs[row, col_pos * 2])
        plot_ax = fig.add_subplot(gs[row, col_pos * 2 + 1], sharey=hist_ax)
        plot_axes.append(plot_ax)
        hist_axes.append(hist_ax)

        if col in df.columns:
            # ---- SCATTER PLOT, COLOURED BY V_MAG GROUPS, palette as in context ----
            # Scatter, each group in its colour. Use a loop for groups.
            for idx in range(N_COLOR_GROUPS):
                group_mask = (df_color_group == idx)
                # Only plot points with both Teff and col present
                mask_finite = (
                    group_mask &
                    np.isfinite(df["T_eff [K]"]) &
                    np.isfinite(df[col])
                )
                if np.any(mask_finite):
                    plot_ax.scatter(
                        df.loc[mask_finite, "T_eff [K]"],
                        df.loc[mask_finite, col],
                        alpha=0.8,
                        color=COLOR_PALETTE[idx % len(COLOR_PALETTE)],
                        s=10,
                        label=None  # No longer put label on any subplot
                    )
            # Only set xlabel if this is the last row (so all subplots in last row get the X label, others do not)
            if row == n_rows - 1:
                plot_ax.set_xlabel(r'$T_\mathrm{eff}$ [K]', fontsize=12)
            else:
                plot_ax.set_xlabel("")  # Remove label, keep ticks

            plot_ax.grid(True, linestyle='--', alpha=0.6)

            # Set xscale and yscale for Distance [pc] plot (log yscale if present)
            if col == "Distance [pc]":
                # plot_ax.set_yscale("log")
                # Optionally: set finite y limits, e.g. to omit extreme outliers
                finite_dist = df["Distance [pc]"][np.isfinite(df["Distance [pc]"])]
                if len(finite_dist) > 0:
                    min_dist = np.nanmax([finite_dist.min(), 0.1])
                    max_dist = np.nanmin([finite_dist.max(), 5e4])
                    plot_ax.set_ylim(min_dist, max_dist)

            # --- Histogram for all points (not group split) ---
            # Only use finite ydata
            valid = np.isfinite(df[col].values)
            ydata = df[col].values[valid]
            bins = min(40, max(10, int(np.sqrt(len(ydata)))))
            try:
                counts, edges = np.histogram(ydata, bins=bins)
                centers = (edges[:-1] + edges[1:]) / 2
                # Mirror: bars go left from zero
                hist_ax.barh(
                    centers,
                    -counts,
                    height=np.diff(edges),
                    color="#377eb8",
                    alpha=0.5,
                    align='center'
                )
                grand_max = counts.max() if len(counts) else 0
                hist_ax.set_xlim(-grand_max * 1.05 if grand_max else -1, 0)
            except Exception:
                hist_ax.set_axis_off()

            # Axis formatting: mostly as before
            hist_ax.yaxis.set_ticks_position('right')
            hist_ax.yaxis.set_label_position('right')
            
            # Remove horizontal part of the axis of histogram
            hist_ax.spines['bottom'].set_visible(False)
            hist_ax.spines['top'].set_visible(False)
            hist_ax.spines['left'].set_visible(False)
            hist_ax.spines['right'].set_visible(True)

            # Remove all ticks and grid from histogram
            hist_ax.tick_params(
                axis='y', labelleft=False, labelright=False, left=False, right=False
            )
            hist_ax.tick_params(
                axis='x', labeltop=False, labelbottom=False, top=False, bottom=False
            )
            plt.setp(hist_ax.get_xticklabels(), visible=False)
            plt.setp(hist_ax.get_yticklabels(), visible=False)
            hist_ax.grid(False)

            plot_ax.yaxis.set_ticks_position('left')
            plot_ax.yaxis.set_label_position('left')
            plot_ax.spines['right'].set_visible(False)
            plot_ax.spines['left'].set_visible(True)
            plot_ax.spines['top'].set_visible(False)
            plot_ax.tick_params(axis='y', labelleft=True, labelright=False, left=True, right=False)

            plot_ax.set_ylabel(ylabels_dict.get(col, col), fontsize=12)

            plt.setp(plot_ax.get_yticklabels(), fontsize=8)

            # Match y-limits for overlay
            plot_ax_ylim = plot_ax.get_ylim()
            hist_ax.set_ylim(plot_ax_ylim)

            plot_ax.yaxis.set_tick_params(labelright=False)
            plot_ax.tick_params(left=True, right=False)

            if row != n_rows - 1:
                plt.setp(hist_ax.get_xticklabels(), visible=False)
                hist_ax.set_xlabel('')

        else:
            plot_ax.set_axis_off()
            hist_ax.set_axis_off()
            plot_ax.text(0.5, 0.5, f'Missing:\n{col}', ha='center', va='center', transform=plot_ax.transAxes)

    # Add a figure-level legend for the V magnitude bins at the top
    handles = []
    labels = []
    for idx in range(N_COLOR_GROUPS):
        # Only add bins for which data exists in at least one column
        color = COLOR_PALETTE[idx % len(COLOR_PALETTE)]
        label_str = f"{vmag_edges[idx]:.2f}≤V<{vmag_edges[idx+1]:.2f}"
        handles.append(Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=8))
        labels.append(label_str)
    # Adjust the "bbox_to_anchor" y-coordinate to move the legend closer to the subplots
    fig.legend(handles, labels, title="V magnitude", loc="upper center", 
               bbox_to_anchor=(0.5, 0.99), ncol=N_COLOR_GROUPS, fontsize=12, title_fontsize=12, frameon=True, framealpha=0.8)

    # Turn off any unused subplots
    total_plots = n_rows * n_cols
    for i in range(len(columns), total_plots):
        row = i // n_cols
        col_pos = i % n_cols
        hist_ax = fig.add_subplot(gs[row, col_pos * 2])
        plot_ax = fig.add_subplot(gs[row, col_pos * 2 + 1])
        hist_ax.set_axis_off()
        plot_ax.set_axis_off()

    # Use tight layout with reduced margin to bring subplots closer to the edge (left/right/top/bottom)
    plt.tight_layout(rect=[0.01, 0.012, 0.99, 0.945])

    plt.savefig(f'{FIGURES_DIRECTORY}stellar_properties_vs_temperature_{detection_limit}.png')
    print(f'Saved {FIGURES_DIRECTORY}stellar_properties_vs_temperature_{detection_limit}.png')
    plt.show() if show_plot else plt.close()


In [228]:
def plot_hr_diagram_with_detection_limit(df, use_filtered_data=True, detection_limit=1.5, dpi=150, show_plot=True):
    print('\nPlotting HR diagram with detection limit')
    """
    Create a Hertzsprung-Russell diagram with stars color-coded by their habitable zone detection limit.
    
    Args:
        df (pd.DataFrame): DataFrame containing stellar data
        use_filtered_data (bool, optional): Whether to filter data based on detection limit. Defaults to True
        detection_limit (float, optional): Upper limit for HZ detection limit filtering. Defaults to 1.5
        dpi (int, optional): DPI for the output figure. Defaults to 150
        
    Returns:
        None
    """
    # Determine which data to plot based on filtering option
    if use_filtered_data:
        data_to_plot = df[df['HZ Detection Limit [M_Earth]'] <= detection_limit]
        color_data = data_to_plot['HZ Detection Limit [M_Earth]']
        colorbar_label = r'HZ Detection Limit [$M_\oplus$]'
        filename = f'{FIGURES_DIRECTORY}HR_diagram_HZ_detection_limit_filtered_{detection_limit}.png'
        print(f'Number of stars with HZ Detection Limit <= {detection_limit}: {len(data_to_plot)}')
        plot_stellar_properties_vs_temperature(data_to_plot, detection_limit, show_plot=show_plot)
    else:
        data_to_plot = df
        color_data = np.minimum(df['HZ Detection Limit [M_Earth]'], 4)
        colorbar_label = r'HZ Detection Limit [$M_\oplus$]'
        filename = f'{FIGURES_DIRECTORY}HR_diagram_HZ_detection_limit.png'

    plt.figure(figsize=(10, 8), dpi=dpi)

    sc = plt.scatter(
        data_to_plot['T_eff [K]'], 
        data_to_plot['Luminosity [L_Sun]'], 
        c=color_data,
        cmap='viridis',
        alpha=0.99, 
        edgecolors='grey',
        linewidths=0.05,
        s=data_to_plot['Radius [R_Sun]'] * 20
    )

    cbar = plt.colorbar(sc, label=colorbar_label)
    sc.set_clim(0, 4)

    plt.xscale('log')
    plt.yscale('log')
    plt.xlim(min(df['T_eff [K]'])-50, max(df['T_eff [K]'])+50)
    plt.ylim(min(df['Luminosity [L_Sun]']), max(df['Luminosity [L_Sun]'])+0.5)
    plt.gca().invert_xaxis()

    plt.xlabel('Effective Temperature [K]')
    plt.ylabel('Luminosity [$L_\odot$]')
    if use_filtered_data:
        plt.title('Hertzsprung-Russell Diagram (' + str(len(data_to_plot)) + ' < ' + str(detection_limit) + ' $M_\\oplus$)')
    else:
        plt.title('Hertzsprung-Russell Diagram')
    
    plt.grid(True, which="both", ls="--", linewidth=0.5)

    plt.savefig(filename)
    print(f'Saved {filename}')
    plt.show() if show_plot else plt.close()


plot_hr_diagram_with_detection_limit(
        df,
        use_filtered_data=detection_limit is not None,
        detection_limit=1
    )


Plotting HR diagram with detection limit
Number of stars with HZ Detection Limit <= 1: 256

Plotting stellar properties vs temperature (Vmag color groups, histograms not group-split)
Saved ../figures/stellar_properties_vs_temperature_1.png
Saved ../figures/HR_diagram_HZ_detection_limit_filtered_1.png
